# Быстрое сжатие векторного пространства

Одна из часто возникающих задач - сжатие векторного пространства с приближённым сохранением некоторых свойств (например, взаимного расстояния между векторами). В некоторых приложениях (например - в обработке естественных языков) исходное пространство имеет очень высокую размерность и оптимальное сжатие (например, методом главных компонент или градиентным спуском) потребует существенного расхода процессорного времени и памяти. 

В данной задаче на примере реальных данных показывается эффективный способ провести такое сжатие с сохранением взаимного расстояния между векторами.

### Исходные данные

Исходные данные для задачи содержатся в файле ppmi_vectors.txt. Это разреженные семантические векторы для 4х групп слов. Они имеют такой вид:
```
Слово_1
(номер координаты 1_1, значение)
(номер координаты 1_2, значение)
...
(номер координаты 1_M1, значение)

Слово_2
(номер координаты 2_1, значение)
(номер координаты 2_2, значение)
...
(номер координаты 2_M2, значение)
```
номера координат неупорядочены, вектора не нормированы. Размерность исходного пространства 1 400 000. Семантические векторы представляют собой столбцы матрицы взаимной информации слов ( [ppmi, positive pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) ). Здесь приведены лишь некоторые столбцы т.к. сама матрица слишком велка для учебной задачи (~ 1 400 000 x 5 000 ~ 20гб). Сама матрица была получена путём вычисления ppmi для слов на 1гб русскоязычных текстов. 

За предоставление данных авторы курса выражают признательность коллективу лаборатории когнитивных архитектур МФТИ и, в частности, Фахрутдинову Тимуру.

Тоже выражаю респект Тимуру, классный чел

### Постановка задачи

Оказывается, если исходные вектора просто умножить на фиксированную случайную матрицу, то расстояния между ними с хорошей точностью сохранятся. Более точно, пусть в исходном пространстве размерности $N$ имелись нормированные вектора $word_1$ и $word_2$. Пусть $R$ - случайная матрица $M$ X $N$, элементы которой равномерно распределены на отрезке $[-1, 1]$. Тогда при достаточно больших M и N имеет место приближённое равенство:

$$ (word_1, word_2) \approx \dfrac{(R \cdot word_1, R \cdot word_2)}{\sqrt{(R \cdot word_1, R \cdot word_1) (R \cdot word_2, R \cdot word_2)}} $$

Или, обозначив нормированные новые вектора 

$$ word_1' := \dfrac{word_1}{\sqrt{(R \cdot word_1, R \cdot word_1)}};\ \ \ word_2' := \dfrac{word_2}{\sqrt{(R \cdot word_2, R \cdot word_2)}}$$

Имеем
$$ (word_1, word_2) \approx  (word_1', word_2') $$

1. Дополните код в ячейках ниже в соответствии с комментариями. Итого вы должны на нескольких примерах продемонстрировать, что при сжатии исходного пространства до размерности 1024 указанным выше способом, "синонимичные" слова (те, нормированные семантические вектора которых близки) остаются "синонимичными", а те, которые "синонимичными" не являлись - не не становятся "синонимами".

2. По нескольким точкам постройте график зависимости ошибки от размерности финального пространства (может потребовать значительного компьютерного времени).

3. (*)Теоретически выведите приближённое равенство $ (word_1, word_2) \approx  (word_1', word_2') $. Какие условия для него нужны и каковы асимптотики ошибки? Сверьте с п. 2 

П.3 не является обязательным.

In [106]:
# Допишите код, который будет загружать данные из файла "ppmi_vectors.txt"
# в словарь words_vectors с разреженными семантическими векторами

# выбираем удобный класс разреженных векторов
# при проблемах с импортом класса - воспользуйтесь рекомендацией по ссылке
# https://github.com/scikit-learn/scikit-learn/issues/16727
from scipy.sparse import dok_array 
import re # для загрузки данных будут удобны регулярные выражения
import numpy as np
vetors_file = open("ppmi_vectors.txt", "r")
vector_space_size = 1400*1000
lines = vetors_file.readlines()

i = 0
words_vectors = dict()
current_word = ""
# Разреженные вектора представляются матрицами размерами Nx1
# Это не то же самое, что векторы длины N
current_vect = dok_array((vector_space_size, 1),dtype = np.float32)
for line in lines:
    i += 1
    word_m = re.match(r"[а-яА-Я]{1,100}", line) # ищем слово
    if word_m is not None: # началось новое слово
        if current_word != "": 
            words_vectors[current_word] = current_vect
        current_word = word_m.group(0) # запомнили новое слово
        # Создаём под него чистый разреженный вектор
        current_vect = dok_array((vector_space_size, 1),dtype = np.float32)

        
    vect_component_m = re.match(r"[\(\)0-9\.\,\s]{1,100}", line)
    if vect_component_m is not None: # обнаружили очередную компоненту вектора
        vect_component = vect_component_m.group(0)
        axis_m  = re.search(r"[0-9]{1,30}", vect_component)
        if axis_m is not None:
            axis    = int(axis_m.group(0)) # записали номер оси
            value_m = re.search(r"[0-9]{1,30}\.[0-9]{1,30}", vect_component)
            if value_m is not None:
                value   = float(value_m.group(0)) # записали значение
                # добавьте значение в словарь

In [107]:
# выведите все слова, представленные в словаре

dict_keys(['Иван', 'Николай', 'Александр', 'Михаил', 'Василий', 'Миллион', 'Миллиард', 'Тысяча', 'Пять', 'Шесть', 'Семь', 'Восемь', 'России', 'РФ', 'СССР', 'Германии', 'ЦСКА', 'Динамо', 'Локомотив'])

In [108]:
# Сохраняться при сжатии пространства будет нормированное скалярное произведение
def normed_dot(words_vectors, word1, word2):
    # Напишите функцию, принимающую на вход словарь разреженных векторов,
    # и два слова, а возвращающую скалярное произведение нормированных векторов

In [121]:
# Во время сжатия вся матрица перехода не хранится в памяти, а генерируется построчно
# т.к. её минимальный объём (float16 size)*vector_space_size*final_space_size = 
# 2*1 024*1 400 000 ~ 3*10^9 байт ~ 3гб (для float32 и final_space_size = 4096 уже 25гб)
final_space_size = 1024 #4096
new_words_vectors = dict()
for word in words_vectors.keys():
    new_words_vectors[word] = np.zeros((final_space_size, 1))
for i in range(final_space_size):
    rand_vect = # задаёте строку матрицы R (случайные числа на отрезке -1, 1)
    for word in words_vectors.keys():
        new_words_vectors[word][i] = # умножьте rand_vect на исходный вектор 
    if (i % 100 == 0):
        print(round(i/final_space_size*100), "% done")

SyntaxError: invalid syntax (2493515654.py, line 9)

In [122]:
# Выведите по крайней мере 4 примера, сравнивающих скалярные произведения слов 
# из разных смысловых групп и продемонстрируйте, что смысловые группы не перемешались

In [123]:
def mean_error(words_vectors, new_words_vectors):
    # Напишите функцию подсчитывающую среднеквадратичную ошибку сжатия пространства 
    # в смысле сохранения скалярного произведения между различными нормированными векторами

SyntaxError: unexpected EOF while parsing (2605310473.py, line 3)

In [120]:
mean_error(words_vectors, new_words_vectors)

0.03176239287802527

In [ ]:
# По нескольким точкам постройте график зависимости этой ошибки от размерности финального пространства